In [22]:
#imports
#Created by Jessica

import os
import json
import tempfile
from openforcefield.topology import Molecule, Topology
from openforcefield.typing.engines.smirnoff import (ForceField,
UnassignedValenceParameterException, BondHandler, AngleHandler,
ProperTorsionHandler, ImproperTorsionHandler,
vdWHandler)
from plot_td_energies import collect_td_targets_data, plot_td_targets_data
import pickle
from make_torsion_target_new import *
import shutil

force_balance_file = 'optimize.in'

In [23]:
#functions
#sub_dir = [x[0] for x in os.walk(directory) if os.path.basename(x[0]).startswith('td')]

def makePlots(ds, ff, param_fit, ff_analysis, param_analysis, target_directory):
    """
    Description: 
    -Create k versus WBO plots
    -Create MM versus QM plots for each target 
    
    
    input:
    ds: List of dataset names
    ff: .offxml style format force field name
    param: List of parameters to generate plots for
    """
    
    #clear out target directory to create new targets based on dataset specifications 
    shutil.rmtree(target_directory)
    os.mkdir(target_directory)
    
    
    #make td Targets 
    for d in ds: 
        torsiondrive_data=download_torsiondrive_data(d)
        make_torsiondrive_target(d, torsiondrive_data, test_ff=None)
        
        

    #Create targets folder in subdirectory 
    force_balance_file = 'optimize.in'


    # Find which line of optimize.in the targets specificaiton begins at
    with open(force_balance_file, 'r') as f:
        replace_index = None
        force_balance_lines = f.readlines()
        for index,line in enumerate(force_balance_lines):
            try:
                #if 'name' in line[:4]:
                if line.startswith('name'):
                    replace_index = index
                    break
            except:
                pass
        #print(replace_index)
    
    count = 0
    for asub in sub_dir:
        #do somehting with metadata
        metadata = os.path.join(asub,'metadata.json')
        #do something with mol2 file
        mol2 = os.path.join(asub, 'input.mol2')
        #get indices from json
        with open(metadata, 'r') as f:
            data = json.load(f)
            #print(data['attributes']['canonical_isomeric_explicit_hydrogen_mapped_smiles'])
            cmiles=data['attributes']['canonical_isomeric_explicit_hydrogen_mapped_smiles']
            dihedral=tuple(data['dihedrals'][0])
            #print(dihedral)

        #molecules =  Molecule.from_file(mol2, allow_undefined_stereo=True)
        molecules=Molecule.from_mapped_smiles(cmiles)
        topology = Topology.from_molecules([molecules])
        molecules.visualize()
        # Let's label using the Parsley force field
        forcefield = ForceField(ff, allow_cosmetic_attributes=True)

        # Run the molecule labeling
        molecule_force_list = forcefield.label_molecules(topology)
        #print(dict(molecule_force_list[0]['ProperTorsions']))
        # Print out a formatted description of the torsion parameters applied to this molecule
        plot_dict = {}
        for mol_idx, mol_forces in enumerate(molecule_force_list):
            for force_tag, force_dict in mol_forces.items():
                if force_tag == 'ProperTorsions':
                    tigcount=0
                    for (atom_indices, parameter) in force_dict.items():
                        #print('param id', parameter.id)
                        if (parameter.id == "TIG-fit0"):
                            tigcount+=1
                    if tigcount==4: 
                        for (atom_indices, parameter) in force_dict.items():
                            #print('param id', parameter.id)
                            if (parameter.id == "TIG-fit0"):
                                #print('check1', parameter.id)
                                #check dihedral in forward or reverse order
                                if (atom_indices == dihedral) or (atom_indices == dihedral[::-1]):
                                    print(atom_indices, dihedral)
                                    print(asub.split('/')[-1])
                                    #paste
                                    print('check2')
                                    #run forcebalance
                                    tmp_file = tempfile.NamedTemporaryFile(suffix='.in')
                                    if 0:
                                        with open(tmp_file.name,'w+t') as f1:
                                            for index,line in enumerate(force_balance_lines):
                                                if index != replace_index:
                                                    print(index, line)
                                                    f1.write(line)
                                                else:
                                                    print('original', line)
                                                    print('replace ', line.split()[0] + ' ' + os.path.basename(asub))
                                                    f1.write(line.split()[0] + ' ' + os.path.basename(asub)+"\n")
                                            f1.flush()
                                            f1.seek(0)
                                            print('ForceBalance ' + f1.name)
                                            print(os.path.isfile(f1.name))
                                            print(os.system('ForceBalance ' +f1.name))
                                            print('done')

                                            import time
                                    if 1:
                                        with open('debug.in','w') as f1:
                                            for index,line in enumerate(force_balance_lines):
                                                if index != replace_index:
                                                    f1.write(line)
                                                else:
                                                    f1.write(line.split()[0] + ' ' + os.path.basename(asub)+"\n")
                                        print('ForceBalance ' + 'debug.in')
                                        print(os.system('ForceBalance ' + 'debug.in'))
                                        print('done')
                                        #make plot
                                        if 0:
                                            plot_data = collect_td_targets_data('debug.tmp', 'targets')
                                            plot_dict.update(plot_data)
                                            plot_td_targets_data(data)
    with open('plot_data.pk', 'wb') as pk:
        pickle.dump(plot_dict,pk)
    

    
    #run force balance 
    #store the k1 and k2 values
    #getKval(k1, k2)=kval
    
    #store data in dictionary data {cmiles : [wbo, kval], }
    
    #create figure 
    
    return 
    
def getKval(k1, k2):
    """
    Determine the slope from the k1 and k2 value
    
    k1: 
    k2:
    """
    
    
    

    return kval 


def make_wbo_versus_k_plots(valDict):
    """
    Description: Make plots of k versus WBO and save plots to .pdf 
    
    
    input: 
    valDict: Dictionary of cmiles as keys and list of [wbo, kval] as the items 
    
    """
    
    
    
    
    return 



def make_MM_QM_plots(datasets):
    """
    datasets: MM_QM_Plots
    
    """

    
    
    
    return 




    




In [24]:
#directory = '/Users/jessica/Downloads/release_1.2.0/fb-fit/targets/'
ff = '/Users/jessica/Documents/Grad_research/WBO_Torsions_ChayaPaper/release_1.3.0_2/fb-fit/fb-fit0/forcefield/test.offxml'

#be very careful when specifying this 
target_directory='/Users/jessica/Documents/Grad_research/wbointerpolation/by_molecule_experiment/fb-fit0/targets'
#all of the molecules there is torsions 
#break them out into subplots for the TIG parameters 


#Plot for molecules that use TIG5a, but use the TIG-fit0 plots 
#Plot for TIG3 

#run the same experiments with Carbon-Nitrogen central bonds parameters 


ds=['SMIRNOFF Coverage Torsion Set 1']
ff='/Users/jessica/Documents/Grad_research/WBO_Torsions_ChayaPaper/release_1.3.0_2/fb-fit/fb-fit0/forcefield/test.offxml'
param_fit=['TIG-fit0'] 
param_analysis=[]
ff_analysis=''

#makePlots(ds, ff, param_fit, ff_analysis, param_analysis, target_directory)








In [25]:
ds = ['OpenFF Substituted Phenyl Set 1']
makePlots(ds, ff, param_fit, ff_analysis, param_analysis, target_directory)


Loading TorsionDrive Scans from [ OpenFF Substituted Phenyl Set 1 ] spec [default]
Found 159 data entries
Found 159 torsiondrive records
    1 : CCC(=O)Nc1[cH:1][c:2](ccn1)[NH:3][CH2:4]C          status COMPLETE
td_record.get_final_molecules() {(-165,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='fe9dd1f'), (165,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='3899fd8'), (-150,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='57e7e5b'), (0,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='3dd3c2b'), (180,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='2ae7c4a'), (-45,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='56b30c1'), (-15,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='be35372'), (15,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='e59764d'), (-60,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='df61219'), (-30,): Molecule(name='C10H15N3O', formula='C10H15N3O', hash='93939fb'), (30,): Molecule(name='C10H

    6 : CCNc1[cH:1][c:2](ccn1)[NH:3][C:4](=O)NC            status COMPLETE
td_record.get_final_molecules() {(0,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='ba629c2'), (15,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='8693b37'), (-165,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='a5ac147'), (135,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='1500dc5'), (30,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='bc3b5d5'), (60,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='ddd4a0d'), (120,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='32d0c07'), (-135,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='a92c413'), (-75,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='8910b22'), (45,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='f83f994'), (75,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='b4bc027'), (105,): Molecule(name='C9H14N4O', formula='C9H14N4O', hash='4bf9859'), (165,): Molecule(name='C9H14N4O', formula='C9

   11 : CCO[C:3](=[O:4])[c:2]1[cH:1]cnc(c1)[N+](C)(C)C     status COMPLETE
td_record.get_final_molecules() {(180,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='f4844fc'), (-165,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='3cf3b5c'), (165,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='7ae90d4'), (-150,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='a649ed0'), (-90,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='6d24a77'), (90,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='3720dde'), (150,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='8c392fe'), (-135,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='c89f68d'), (-105,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='fddbc2c'), (105,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='df6630d'), (135,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash='8286fbd'), (-120,): Molecule(name='C11H17N2O2', formula='C11H17N2O2', hash=

   16 : CC[C:4](=O)[NH:3][c:2]1[cH:1]cc(cc1)F              status COMPLETE
td_record.get_final_molecules() {(-90,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='8e92d80'), (120,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='8f3c844'), (150,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='199ed54'), (-105,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='a5b1d31'), (135,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='808aedd'), (90,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='216ab1a'), (180,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='7d82b6d'), (-165,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='082bf4e'), (-135,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='11c9000'), (-75,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='366451d'), (75,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='a1088a3'), (105,): Molecule(name='C9FH10NO', formula='C9FH10NO', hash='6f10a10'), (165,): Molecule(name='C9FH10NO', formu

   21 : CC[C:4](=O)[NH:3][c:2]1ccc(n[cH:1]1)[N+](C)(C)C    status COMPLETE
td_record.get_final_molecules() {(-165,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='1891b31'), (0,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='5b4856a'), (180,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='f503a41'), (-135,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='d081e46'), (-15,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='3af5506'), (15,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='b110c10'), (165,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='f877378'), (-150,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='b7157a7'), (150,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='8ee36d5'), (-105,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='8fbc9ee'), (-45,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='9917528'), (45,): Molecule(name='C11H18N3O', formula='C11H18N3O', hash='38e347d'), (75,): Molecule(

...

  149 : c1[cH:1][c:2](c(cc1O)F)[c:4]2[cH:3]c[nH+]cc2       status COMPLETE
td_record.get_final_molecules() {(-135,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='1e2078b'), (-105,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='8daca6b'), (-120,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='c42bfdc'), (-165,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='10b8e0c'), (-150,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='32c82c9'), (-90,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='d995cda'), (180,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='15943d0'), (-75,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='c013ab2'), (-45,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='2fcc079'), (165,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='1cb20d6'), (-60,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='2c23ce8'), (150,): Molecule(name='C11FH9NO', formula='C11FH9NO', hash='1a0ee84'), (-15,): Molecule(name='C11FH9NO', f

  154 : c1ccc([c:2]([cH:1]1)[c:3]2[cH:4]cncc2)F            status COMPLETE
td_record.get_final_molecules() {(-135,): Molecule(name='C11FH8N', formula='C11FH8N', hash='a58c56c'), (-105,): Molecule(name='C11FH8N', formula='C11FH8N', hash='c670d93'), (-120,): Molecule(name='C11FH8N', formula='C11FH8N', hash='08684be'), (-90,): Molecule(name='C11FH8N', formula='C11FH8N', hash='0865bed'), (-150,): Molecule(name='C11FH8N', formula='C11FH8N', hash='95c2f0d'), (-60,): Molecule(name='C11FH8N', formula='C11FH8N', hash='c0eec07'), (180,): Molecule(name='C11FH8N', formula='C11FH8N', hash='634cd33'), (-165,): Molecule(name='C11FH8N', formula='C11FH8N', hash='57b5b3e'), (-75,): Molecule(name='C11FH8N', formula='C11FH8N', hash='2f401a5'), (165,): Molecule(name='C11FH8N', formula='C11FH8N', hash='c13f764'), (150,): Molecule(name='C11FH8N', formula='C11FH8N', hash='d60578a'), (-45,): Molecule(name='C11FH8N', formula='C11FH8N', hash='8b48d2e'), (-15,): Molecule(name='C11FH8N', formula='C11FH8N', hash='8

  159 : c1ccc([c:2]([cH:1]1)[c:3]2[cH:4]cncc2)Cl           status COMPLETE
td_record.get_final_molecules() {(-90,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='4aa00d5'), (-105,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='dd65072'), (-75,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='72891c2'), (-120,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='4574337'), (-60,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='17c956e'), (-30,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='8f48dff'), (-165,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='770a06b'), (-135,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='39670f2'), (-45,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='3a884a5'), (-15,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='041a9d2'), (-150,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='e32ce1b'), (0,): Molecule(name='C11ClH8N', formula='C11ClH8N', hash='f6d1e1f'), (180,): Molecule(name='C11ClH8N', for

122: td_OpenFF_Substituted_Phenyl_Set_1_122_C7H8O
123: td_OpenFF_Substituted_Phenyl_Set_1_123_C5H6N2O
124: td_OpenFF_Substituted_Phenyl_Set_1_124_C8H11N3O2
125: td_OpenFF_Substituted_Phenyl_Set_1_125_C8H13N2O
126: td_OpenFF_Substituted_Phenyl_Set_1_126_C8H11NO
127: td_OpenFF_Substituted_Phenyl_Set_1_127_C8H13N2O
128: td_OpenFF_Substituted_Phenyl_Set_1_128_C9H14NO
129: td_OpenFF_Substituted_Phenyl_Set_1_129_C8H13N2O
130: td_OpenFF_Substituted_Phenyl_Set_1_130_C7H8N4O3
131: td_OpenFF_Substituted_Phenyl_Set_1_131_C7H4N2O2
132: td_OpenFF_Substituted_Phenyl_Set_1_132_C5FH3N2O2
133: td_OpenFF_Substituted_Phenyl_Set_1_133_C6H6N4O3
134: td_OpenFF_Substituted_Phenyl_Set_1_134_C6H6N4O3
135: td_OpenFF_Substituted_Phenyl_Set_1_135_C6H5N3O4
136: td_OpenFF_Substituted_Phenyl_Set_1_136_C5H5N3O2
137: td_OpenFF_Substituted_Phenyl_Set_1_137_C6H6N4O3
138: td_OpenFF_Substituted_Phenyl_Set_1_138_C8H8N2O3
139: td_OpenFF_Substituted_Phenyl_Set_1_139_C8H8N2O3
140: td_OpenFF_Substituted_Phenyl_Set_1_140_C7H7N3

NameError: name 'sub_dir' is not defined